In [93]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from pathlib import Path
from langchain.docstore.document import Document
import torch 
import numpy as np
load_dotenv()

True

In [94]:
os.environ['LANGCHAIN_TRACING_V2'] = os.getenv('LANGCHAIN_TRACING_V2')
os.environ['LANGCHAIN_ENDPOINT'] = os.getenv('LANGCHAIN_ENDPOINT')
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_PROJECT'] = os.getenv('LANGCHAIN_PROJECT')
os.environ['TAVILY_API_KEY'] = os.getenv('TAVILY_API_KEY')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY')
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')


In [95]:
class Utf8TextLoader(TextLoader):
    def __init__(self, file_path):
        self.file_path = file_path

    def load(self):
        with open(self.file_path, 'r', encoding='utf-8') as f:
            return f.read()


def read_text_file(file_path):
    return Path(file_path).read_text(encoding='utf-8')


def create_documents_from_files(file_paths):
    documents = []
    for file_path in file_paths:
        content = read_text_file(file_path)
        documents.append(Document(page_content=content))
    return documents

In [96]:
file_paths = list(Path('./source').glob("*.txt"))
docs = create_documents_from_files(file_paths)

In [97]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=40,
    length_function=len,
)

In [98]:
new_docs = text_splitter.split_documents(documents=docs)
doc_strings = [{"page_content": doc.page_content} for doc in new_docs]
print(doc_strings)

[{'page_content': 'El deporte es una actividad física que desempeña un papel crucial en la vida de las personas en todo el mundo. No solo es una fuente de entretenimiento y competencia, sino que también promueve la salud física y mental, fomenta el trabajo en equipo y contribuye al desarrollo de valores sociales y culturales. La diversidad de disciplinas deportivas, desde el fútbol y el baloncesto hasta el atletismo'}, {'page_content': 'y el baloncesto hasta el atletismo y la natación, ofrece opciones para todos los gustos y habilidades, haciendo del deporte una parte integral de la sociedad moderna.'}, {'page_content': 'Uno de los deportes más populares a nivel global es el fútbol. Conocido como "el deporte rey", el fútbol se juega y se sigue fervientemente en todos los continentes. La Copa Mundial de la FIFA, celebrada cada cuatro años, es uno de los eventos deportivos más vistos en el planeta, uniendo a millones de aficionados en una celebración de habilidad, estrategia y pasión. La

In [99]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [100]:
print(f"Using device: {device}")

Using device: cpu


In [101]:
class VectorStoreRetriever:
    def __init__(self, docs: list, vectors: list, oai_client):
        self._arr = np.array(vectors)
        self._docs = docs
        self._client = oai_client

    @classmethod
    def from_docs(cls, docs, oai_client):
        embeddings = oai_client.embed_documents([doc["page_content"] for doc in docs])
        vectors = embeddings
        return cls(docs, vectors, oai_client)

    def query(self, query: str, k: int = 5) -> list[dict]:
        embed = self._client.embed_query(query)
        # "@" is just a matrix multiplication in python
        scores = np.array(embed) @ self._arr.T
        top_k_idx = np.argpartition(scores, -k)[-k:]
        top_k_idx_sorted = top_k_idx[np.argsort(-scores[top_k_idx])]
        return [
            {**self._docs[idx], "similarity": scores[idx]} for idx in top_k_idx_sorted
        ]

In [102]:
embeddings  = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2"
)

d:\Proyecto_holagrama_IA\LanGraph\v2email\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [103]:
retriever = VectorStoreRetriever.from_docs(doc_strings, embeddings)
retriever

In [104]:
resultado =retriever.query(query="Cuentame sobre la cultura japonesa")
resultado

[{'page_content': 'La cultura japonesa es un fascinante contraste entre la tradición y la modernidad. Por un lado, el país es famoso por sus antiguas prácticas y costumbres, como el arte del té, la ceremonia del kimono, y los jardines zen. Lugares emblemáticos como el Templo Senso-ji en Tokio, el Santuario Fushimi Inari-taisha en Kioto y el Castillo de Himeji son testamentos de su herencia histórica y',
  'similarity': 0.7457335445106505},
 {'page_content': 'En términos de educación y ética laboral, Japón es conocido por su riguroso sistema educativo y su cultura de trabajo dedicada. La disciplina y el respeto son valores profundamente arraigados en la sociedad japonesa, lo que se refleja en su sistema educativo y en su enfoque hacia el trabajo.',
  'similarity': 0.6147988173149525},
 {'page_content': 'como hanami, que atrae a millones de visitantes.',
  'similarity': 0.6093569503833363},
 {'page_content': 'En resumen, Japón es una nación de contrastes y armonía, donde la antigua cultu

En extensión .py funciona bien y da el formato bien